<a href="https://colab.research.google.com/github/CarolDekker/EstudosPython/blob/main/Trabalho_Final_DataVis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [150]:
!pip install selenium

In [151]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

**IMPORTAR DATASETS**

In [152]:
df_chegadas2019 = pd.read_csv("/content/drive/MyDrive/Trabalho Datavis/chegadas_2019.csv", encoding = 'latin-1', sep = ";")

In [153]:
df_chegadas2020  = pd.read_csv("/content/drive/MyDrive/Trabalho Datavis/chegadas_2020.csv", encoding = 'latin-1', sep = ";")

In [154]:
df_chegadas2021 = pd.read_csv("/content/drive/MyDrive/Trabalho Datavis/chegadas_2021.csv", encoding = 'latin-1', sep = ";")

In [155]:
df_chegadas2022 = pd.read_csv("/content/drive/MyDrive/Trabalho Datavis/chegadas_2022.csv", encoding = 'latin-1', sep = ";")

In [156]:
df_categorizacao = pd.read_excel("/content/drive/MyDrive/Trabalho Datavis/relatorio_categorizacao_2019-portal.xls")

## **DATASET CATEGORIZAÇÃO**

In [157]:
df_categorizacao.drop_duplicates(inplace=True)

In [158]:
print(df_categorizacao.head())

          MACRO  UF                        REGIAO_TURISTICA   MUNICIPIO  \
0  Centro-Oeste  MT  Região Turística Portal do Agronegócio  Nova Mutum   
1      Nordeste  MA            Polo Lagos e Campos Floridos     Cajapió   
2       Sudeste  MG                 Caminhos da Mantiqueira  Brazópolis   
3       Sudeste  MG                                    Ouro       Caeté   
4       Sudeste  MG                        Pedras Preciosas      Catuji   

   COD_IBGE  QUANTIDADE_EMPREGOS  QUANTIDADE_ ESTABELECIMENTOS  \
0   5106224                   89                            14   
1   2102408                    0                             0   
2   3108909                    6                             3   
3   3110004                  310                            11   
4   3115458                    0                             0   

   QUANTIDADE_VISITAS_ESTIMADAS_ INTERNACIONAL  \
0                                            0   
1                                            0   
2 

In [159]:
geolocator = Nominatim(user_agent="my_geocoder")

In [160]:
def get_coordinates(cidade):
  try:
        location = geolocator.geocode(cidade, timeout=10)
        if location:
            return [location.latitude, location.longitude]
  except  GeocoderTimedOut:
        print(f"Timeout ao obter coordenadas para {cidade}. Tentando novamente...")
        time.sleep(2)
        return get_coordinates(cidade)
  except Exception as e:
        print(f"Erro ao obter coordenadas para {cidade}: {str(e)}")
  return [None, None]

In [161]:
df_categorizacao['Coordinates'] = df_categorizacao['MUNICIPIO'].apply(get_coordinates)


NameError: ignored

In [162]:
df_categorizacao[['Latitude', 'Longitude']] = pd.DataFrame(df_categorizacao['Coordinates'].tolist(), index=df_categorizacao.index)

KeyError: ignored

In [ ]:
df_categorizacao = df_categorizacao.dropna(subset=['Latitude', 'Longitude'])

In [ ]:
time.sleep(1)

In [ ]:
print(df_categorizacao.head())

In [ ]:
heat_data = [[row['Latitude'], row['Longitude'], row['QUANTIDADE_VISITAS_ESTIMADAS_ INTERNACIONAL']] for index, row in df_categorizacao.iterrows()]
mapa = folium.Map(location=[df_categorizacao['Latitude'].mean(), df_categorizacao['Longitude'].mean()], zoom_start=10)
HeatMap(heat_data, name='Quantidade de Visitas Estimadas Internacional', radius=15, blur=20).add_to(mapa)

In [ ]:
# Adicionando informações adicionais como popups
for index, row in df_categorizacao.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']],
                  popup=f'Município: {row["MUNICIPIO"]}\nQuantidade de Empregos: {row["QUANTIDADE_EMPREGOS"]}\nArrecadação: {row["ARRECADACAO"]}').add_to(mapa)


In [ ]:
# Adicionando legenda
folium.LayerControl().add_to(mapa)

In [ ]:
# Salvar o mapa de calor como um arquivo HTML
html_mapa = "/content/drive/MyDrive/Trabalho Datavis/mapa_de_calor.html"
mapa.save(html_mapa)

In [ ]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
driver.get(f'file://{html_mapa}')

In [ ]:
driver.save_screenshot("/content/drive/MyDrive/Trabalho Datavis/mapa_de_calor.png")
driver.quit()